<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/screen_align.png" style="margin: 0 auto;">


<h1 style="text-align: center; font-size=58px;">Robust Client Configuration</h1>

In this lesson, we're going to discuss ways in which you can make your application's configuration more robust, with respect to how it talks to the database.

## Always use Connection Pooling

You've learned about connection pooling already but it's important so we'll briefly cover it again.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_connection_pooling_three_connections.png" style="margin: 0 auto;">

Creating a new MongoClient for every request to the database might service your application in the short term, but it will eventually result in the application consuming and depleting available resources that become more and more scarce over time.

Connection pooling reduces the total overhead associated with creating a new connection, by allowing the application to recycle and reuse database connections for new requests.

The M220 API that you've been given correctly reuses the same class or object for all client communication if you'd like to look at an example of how we did it.

# Always specify a `wtimeout` with majority writes.

Another way to make a more robust database client is a write timeout, or `wtimeout`.

No matter how well we engineer a system, we should always expect application external resources like queues, networks, and databases to take more time than expected. For application or consumer critical operations, a developer may choose to write with `w: majority` to ensure that acknowledged writes are written to a majority of replica set nodes.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_primary_secondary_highlighted_w_majority.png" style="margin: 0 auto;">

But if there's a problem on the secondary nodes, we might not get acknowledgements back from the server for a while. If more writes than reads are coming into the system and operations aren't being acknowledged, this will eventually lead to system gridlock.

To avoid this, follow a simple rule. For any write operation written with majority, always specify a write timeout. The specific length of the timeout will need to be determined based on your network and hardware, but you should always be setting timeouts on these (point) writes.

<h1 style="text-align: center;"><code>{ w: "majority", wtimeout: 5000 }</code></h1>

# Always configure for and handle `serverSelectionTimeout` errors.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/world_map.png" style="margin: 0 auto;">

And lastly, you should always handle a server selection timeout error.

This error will be thrown in the event a MongoDB server is unavailable for a write or for a read with a preference that the replica set can't currently fulfill. At the end of the day, MongoDB is a distributed database. So you should expect the system to be running on remote servers, along with all the benefits and constraints that it brings to your application logic.

By default, the time before a driver will raise this error is 30 seconds, but you should change this to suit your application's needs. By handling this error you also passively monitor the health of your application stack and can become very quickly aware of any hardware and software problems that haven't recovered in an adequate amount of time.

Each driver and programming language has a specific way to deal with errors, and we handle this error in particular in Mflix.

## Summary

- Always use connection pooling
- Always specify a `wtimeout` with majority writes
- Always handle `serverSelectionTimeout` errors.